# TensorFlow Tutorial #02
# Convolutional Neural Network

by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)
/ [GitHub](https://github.com/Hvass-Labs/TensorFlow-Tutorials) / [Videos on YouTube](https://www.youtube.com/playlist?list=PL9Hr9sNUjfsmEu1ZniY0XpHSzl5uihcXZ)

## Imports

In [1]:
%matplotlib inline
from datetime import timedelta
import math
import cv2
import numpy as np
import os,datetime,time , sys
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
from scipy.misc import  imread,imresize
import operator
from PIL import ImageGrab
import time
from time import sleep
print ("Package loaded") 

Package loaded


This was developed using Python 3.5.2 (Anaconda) and TensorFlow version:

In [2]:
tf.__version__

'1.2.0'

## Configuration of Neural Network

The configuration of the Convolutional Neural Network is defined here for convenience, so you can easily find and change these numbers and re-run the Notebook.

In [3]:
# Convolutional Layer 1.
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 5          # Convolution filters are 5 x 5 pixels.
num_filters2 = 36         # There are 36 of these filters.

# Fully-connected layer.
fc_size = 128             # Number of neurons in fully-connected layer.

## Data Dimensions

The data dimensions are used in several places in the source-code below. They are defined once so we can use these variables instead of numbers throughout the source-code below.

In [4]:
# We know that MNIST images are 28 pixels in each dimension.
img_size = 64

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = 1

# Number of classes, one class for each of 10 digits.
num_classes = 2

### Helper-function for plotting images

In [5]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

Function used to plot 9 images in a 3x3 grid, and writing the true and predicted classes below each image.

In [6]:
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

### Helper-function for creating a new Convolutional Layer

In [7]:
def new_conv_layer(input,              # The previous layer.
                   num_input_channels, # Num. channels in prev. layer.
                   filter_size,        # Width and height of each filter.
                   num_filters,        # Number of filters.
                   use_pooling=True):  # Use 2x2 max-pooling.

    # Shape of the filter-weights for the convolution.
    # This format is determined by the TensorFlow API.
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # Create new weights aka. filters with the given shape.
    weights = new_weights(shape=shape)

    # Create new biases, one for each filter.
    biases = new_biases(length=num_filters)

    # Create the TensorFlow operation for convolution.
    # Note the strides are set to 1 in all dimensions.
    # The first and last stride must always be 1,
    # because the first is for the image-number and
    # the last is for the input-channel.
    # But e.g. strides=[1, 2, 2, 1] would mean that the filter
    # is moved 2 pixels across the x- and y-axis of the image.
    # The padding is set to 'SAME' which means the input image
    # is padded with zeroes so the size of the output is the same.
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    # Add the biases to the results of the convolution.
    # A bias-value is added to each filter-channel.
    layer += biases

    # Use pooling to down-sample the image resolution?
    if use_pooling:
        # This is 2x2 max-pooling, which means that we
        # consider 2x2 windows and select the largest value
        # in each window. Then we move 2 pixels to the next window.
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    # Rectified Linear Unit (ReLU).
    # It calculates max(x, 0) for each input pixel x.
    # This adds some non-linearity to the formula and allows us
    # to learn more complicated functions.
    layer = tf.nn.relu(layer)

    # Note that ReLU is normally executed before the pooling,
    # but since relu(max_pool(x)) == max_pool(relu(x)) we can
    # save 75% of the relu-operations by max-pooling first.

    # We return both the resulting layer and the filter-weights
    # because we will plot the weights later.
    return layer, weights

### Helper-function for flattening a layer

A convolutional layer produces an output tensor with 4 dimensions. We will add fully-connected layers after the convolution layers, so we need to reduce the 4-dim tensor to 2-dim which can be used as input to the fully-connected layer.

In [8]:
def flatten_layer(layer):
    # Get the shape of the input layer.
    layer_shape = layer.get_shape()

    # The shape of the input layer is assumed to be:
    # layer_shape == [num_images, img_height, img_width, num_channels]

    # The number of features is: img_height * img_width * num_channels
    # We can use a function from TensorFlow to calculate this.
    num_features = layer_shape[1:4].num_elements()
    
    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    layer_flat = tf.reshape(layer, [-1, num_features])

    # The shape of the flattened layer is now:
    # [num_images, img_height * img_width * num_channels]

    # Return both the flattened layer and the number of features.
    return layer_flat, num_features

### Helper-function for creating a new Fully-Connected Layer

In [9]:
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True): # Use Rectified Linear Unit (ReLU)?

    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

### Placeholder variables

In [10]:
X = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')

In [11]:
x_image = tf.reshape(X, [-1, img_size, img_size, num_channels])

In [12]:
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

In [13]:
y_true_cls = tf.argmax(y_true, dimension=1)

### Convolutional Layer 1

Create the first convolutional layer. It takes `x_image` as input and creates `num_filters1` different filters, each having width and height equal to `filter_size1`. Finally we wish to down-sample the image so it is half the size by using 2x2 max-pooling.

In [14]:
layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   use_pooling=True)

Check the shape of the tensor that will be output by the convolutional layer. It is (?, 14, 14, 16) which means that there is an arbitrary number of images (this is the ?), each image is 14 pixels wide and 14 pixels high, and there are 16 different channels, one channel for each of the filters.

### Convolutional Layer 2

Create the second convolutional layer, which takes as input the output from the first convolutional layer. The number of input channels corresponds to the number of filters in the first convolutional layer.

In [15]:
layer_conv2, weights_conv2 = \
    new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=True)

Check the shape of the tensor that will be output from this convolutional layer. The shape is (?, 7, 7, 36) where the ? again means that there is an arbitrary number of images, with each image having width and height of 7 pixels, and there are 36 channels, one for each filter.

### Flatten Layer

The convolutional layers output 4-dim tensors. We now wish to use these as input in a fully-connected network, which requires for the tensors to be reshaped or flattened to 2-dim tensors.

In [16]:
layer_flat, num_features = flatten_layer(layer_conv2)

Check that the tensors now have shape (?, 1764) which means there's an arbitrary number of images which have been flattened to vectors of length 1764 each. Note that 1764 = 7 x 7 x 36.

### Fully-Connected Layer 1

Add a fully-connected layer to the network. The input is the flattened layer from the previous convolution. The number of neurons or nodes in the fully-connected layer is `fc_size`. ReLU is used so we can learn non-linear relations.

In [17]:
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         use_relu=True)

### Fully-Connected Layer 2

Add another fully-connected layer that outputs vectors of length 10 for determining which of the 10 classes the input image belongs to. Note that ReLU is not used in this layer.

In [18]:
layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         use_relu=False)

## TensorFlow Run

### Create TensorFlow session

Once the TensorFlow graph has been created, we have to create a TensorFlow session which is used to execute the graph.

In [19]:
y_pred = tf.nn.softmax(layer_fc2)

In [20]:
session = tf.Session()

In [21]:
layer_fc2

<tf.Tensor 'add_3:0' shape=(?, 2) dtype=float32>

### Initialize variables

The variables for `weights` and `biases` must be initialized before we start optimizing them.

In [22]:
y_pred_cls = tf.argmax(y_pred, dimension=1)

In [23]:
session.run(tf.global_variables_initializer())

### Helper-function to perform optimization iterations

## Load saved weights and biases

In [24]:
#Load weights from saver 
saver = tf.train.Saver(max_to_keep=3) 
saver.restore(session, "C:/Users/nur20/Documents/GitHub/TensorFlow-Master/save/custom_basic_cnn_idle.ckpt")
print("Model restored.")


INFO:tensorflow:Restoring parameters from C:/Users/nur20/Documents/GitHub/TensorFlow-Master/save/custom_basic_cnn_idle.ckpt
Model restored.


In [25]:
# Function, that turnes the image to a gray image
def rgb2gray(rgb):
    # If the length of the images shape is 3
    if len(rgb.shape) is 3:
        return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])
        print("yes")
    else:
        print("no")
        # print ("Current Image if GRAY!")
        return rgb

# Capture Video using Webcam
cap = cv2.VideoCapture(0)

# Load CascadeClassifier to detect Faces -> frontalface.default.xml file)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

font = cv2.FONT_HERSHEY_SIMPLEX

imageCounter = 0
saveCount = 0
piclimit = 3000
# Set the imagesize of the training and testting data
imgsize = [64, 64]
x = 100
y = 100
while (True):
    # capture Frame by Frame
   # ret, frame = cap.read()

    # convert frame to frame_gray -> Frame loses its colore and gets gray
   # frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect face in the gray frame
  #  faces = face_cascade.detectMultiScale(frame)

    
    
    
    img = ImageGrab.grab(bbox = (0,120,1100,750))
    img_np = np.array(img)
    frame = cv2. cvtColor(img_np, cv2.COLOR_BGR2RGB)
    
    #Detect face in the gray frame
    faces=face_cascade.detectMultiScale(frame)

    
    
    faceCaptured = 0
    # defines a list called crop_img
    crop_img = [None]

    imageCounter += 1
    faceCaptured=0

    for (x, y, w, h) in faces:
        if ((w > 100) and (h > 100)):
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            faceCaptured += 1
            crop_img.append(cv2.resize(frame[y + 2:y + h - 1, x + 2:x + w - 1], (250, 250)))

            

            
            
    if(faceCaptured>0):
        grayimg = rgb2gray(crop_img[1])
        graysmall = cv2.resize(grayimg, (imgsize[0], imgsize[1]))/255.
        # reshape image to 1 x 4096 vector
        grayvec   = np.reshape(graysmall, (1, -1))
        prediction = session.run(tf.nn.softmax(y_pred), feed_dict = {X: grayvec})
        index, value = max(enumerate(prediction[0]), key=operator.itemgetter(1)) #find highest value in output vector
        cv2.imshow('grayvec', graysmall)
    else:
        index = 4
        value = 0
    
    className=""
    if index==0:
        className="Casey"
    elif index ==1:
        className="Siraj"
    elif index ==2:
        className="Soeren"
    elif index ==3:
        className="Manu"
    elif index ==4:
        className="Not found"

    print("Prediciton is class '%s' with accuracy %0.3f" % (className, value))

    if index == 0:
        cv2.putText(frame, className, (x, y - 8), font, 3, color=(0, 0, 255),thickness=2)  # Draw the text
    elif index == 1:
        cv2.putText(frame, className, (x, y - 8), font, 3, color=(255, 0, 0),thickness=2)  # Draw the text
    elif index == 2:
        cv2.putText(frame, className, (x, y - 8), font, 2, color=(255, 255, 255))  # Draw the text
    elif index == 3:
        cv2.putText(frame, className, (x, y - 8), font, 2, color=(255, 255, 255))  # Draw the text
    elif index == 4:
        cv2.putText(frame, className, (x, y - 8), font, 2, color=(255, 255, 255))  # Draw the text

    if(faceCaptured>0):
        cv2.putText(frame, str(round(prediction[0][0],2)), (2, 30), font, 1, color=(0, 0, 255))  # Draw the text
        cv2.putText(frame, str(round(prediction[0][1],2)), (2, 60), font, 1, color=(0, 0, 255))  # Draw the text
        #cv2.putText(frame, str(round(prediction[0][2],2)), (2, 90), font, 1, color=(0, 0, 255))  # Draw the text
        #cv2.putText(frame, str(round(prediction[0][3],2)), (2, 120), font, 1, color=(0, 0, 255))  # Draw the text

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindow()


Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Sir

Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton i

Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.719
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with

Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with

Prediciton is class 'Siraj' with accuracy 0.709
Prediciton is class 'Siraj' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Siraj' with accuracy 0.501
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.676
Prediciton is class 'Casey' with accuracy 0.709
Prediciton is class 'Casey' with accuracy 0.731
Prediciton is class 'Casey' with accuracy 0.682
Prediciton is class 'Casey' with accuracy 0.595
Prediciton is class 'Casey' with accuracy 0.567
Prediciton is class 'Casey' with accuracy 0.622
Prediciton is class 'Casey' with accuracy 0.685
Prediciton is class 'Casey' with accuracy 0.680
Prediciton is class 'Casey' with accuracy 0.692
Prediciton is class 'Casey' with accuracy 0.708
Prediciton is class 'Casey' with accurac

...

Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accurac

Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accurac

Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accurac

Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Siraj' with accuracy 0.730
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Prediciton is class 'Not found' with accuracy 0.000
Predicit

KeyboardInterrupt: 

## License (MIT)

Copyright (c) 2016 by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.